## Détection de Fake News avec BERT et les Transformers

Ce projet utilise le modèle de langage BERT et l'architecture Transformer pour créer un modèle de classification capable de distinguer les articles vrais des articles faux. Le dataset utilisé contient deux ensembles de données représentant respectivement les articles véridiques et les fake news. 

### Objectifs
1. Prétraiter les données textuelles pour les adapter aux modèles basés sur des Transformers.
2. Utiliser un modèle BERT pré-entraîné pour encoder les textes en vecteurs représentatifs.
3. Entraîner un modèle de classification pour distinguer les articles en vraies et fausses nouvelles.
4. Évaluer la performance du modèle à l'aide de métriques de classification standard.

Dans ce notebook, nous allons procéder étape par étape pour réaliser cet objectif.


### Installation des bibliothèques nécessaires

Cette cellule installe les bibliothèques `pandas`, `torch`, et `transformers` nécessaires pour le traitement du langage naturel avec le modèle BERT.

#### Étapes :
1. Installation de `pandas` pour la manipulation des données.
2. Installation de `transformers` de Hugging Face pour utiliser des modèles de langage avancés.
3. Installation de `torch` pour la prise en charge de PyTorch, essentiel pour les opérations de calcul du modèle.


In [1]:
!pip install pandas

In [2]:
!pip install transformers

In [3]:
!pip install torch

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import torch
import transformers as transf
import warnings
warnings.filterwarnings('ignore')

c:\Users\deded\Documents\Inge3\NLP\Project\envFakeNews\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Chargement des bibliothèques et des données

Dans cette cellule, nous chargeons deux jeux de données de nouvelles au format CSV, `Fake.csv` et `True.csv`, contenant respectivement des articles de fausses nouvelles et de vraies nouvelles. 

1. **Lecture des fichiers CSV** : 
   - Nous utilisons `pd.read_csv()` pour charger les fichiers dans deux DataFrames : `fake_news_df` et `real_news_df`.
   
2. **Ajout d'une colonne de label** :
   - Une colonne `label` est ajoutée pour chaque dataset : `0` pour les fausses nouvelles, `1` pour les vraies nouvelles, ce qui facilitera la distinction entre les deux classes plus tard.
   
3. **Filtrage des articles courts** :
   - Afin de simplifier le modèle, nous filtrons les articles dont le texte est inférieur à `60 mots`.
   
4. **Équilibrage des classes** :
   - Pour éviter un biais de classe, on sélectionne un nombre égal d'articles dans chaque catégorie (classe minoritaire). Cette opération se fait en prenant l'échantillon `min_class_size` avec `random_state=42` pour garantir la reproductibilité.
   
5. **Fusion et mélange des données** :
   - Les deux DataFrames sont fusionnés en un DataFrame `combined_news_df` et les lignes sont mélangées pour éviter tout biais de séquençage.
   
Enfin, nous affichons quelques lignes de données pour une première vérification visuelle du dataset combiné.



In [ ]:
import pandas as pd

# Charger les fichiers CSV
fake_news_path = "Fake.csv"
real_news_path = "True.csv"

# Lire les deux fichiers
fake_news_df = pd.read_csv(fake_news_path)
real_news_df = pd.read_csv(real_news_path)

# Ajouter une colonne "label" à chaque dataset
fake_news_df['label'] = 0  # Faux articles
real_news_df['label'] = 1  # Vrais articles

# Filtrer les articles avec une longueur de texte inférieure à 50 mots
max_words = 60
fake_news_df = fake_news_df[fake_news_df['text'].apply(lambda x: len(x.split()) < max_words)]
real_news_df = real_news_df[real_news_df['text'].apply(lambda x: len(x.split()) < max_words)]

# Équilibrer les classes en sélectionnant un nombre égal d'articles
min_class_size = min(len(fake_news_df), len(real_news_df))  # Nombre minimal entre les deux classes

fake_news_df_balanced = fake_news_df.sample(n=min_class_size, random_state=42)
real_news_df_balanced = real_news_df.sample(n=min_class_size, random_state=42)

# Fusionner les deux datasets équilibrés
combined_news_df = pd.concat([fake_news_df_balanced, real_news_df_balanced], ignore_index=True)

# Mélanger les données pour éviter l'ordre initial
combined_news_df = combined_news_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Afficher quelques lignes du dataset combiné pour vérifier
combined_news_df.head(-1)


,title,text,subject,date,label
0,LIZ WARREN CALLED OUT For Crazy Claim Steve Ba...,Liberals internal struggle in one clip: .@and...,left-news,"Dec 1, 2016",0
1,No. 2 House Republican says healthcare bill de...,WASHINGTON (Reuters) - The No. 2 Republican in...,politicsNews,"March 23, 2017",1
2,Catalan government says 90 percent voted to le...,MADRID (Reuters) - The Catalan government said...,worldnews,"October 1, 2017",1
3,MIND-BLOWING INTERACTIVE MAP Shows Where Musli...,This is a great visual to share with people wh...,left-news,"Oct 29, 2015",0
4,Trump to meet with Senate Republican leader Mc...,WASHINGTON (Reuters) - U.S. Republican preside...,politicsNews,"May 9, 2016",1


### Distribution des Labels

Cette cellule permet de vérifier que le dataset est bien équilibré. La méthode `value_counts()` est appliquée à la colonne `label` du DataFrame combiné, `combined_news_df`, pour compter le nombre de faux et de vrais articles. Le résultat confirme que chaque classe a un nombre égal d'échantillons, ce qui est crucial pour un apprentissage supervisé équilibré.

In [6]:
label_counts =combined_news_df['label'].value_counts()
label_counts

label
0    679
1    679
Name: count, dtype: int64

### Chargement du Modèle de Transformer et du Tokenizer

Nous définissons ici le modèle de Transformer et son tokenizer. 

1. **Choix du modèle DistilBERT** : 
   - Nous choisissons `DistilBERT`, une version allégée de BERT, pour des raisons de performance tout en maintenant une qualité acceptable.
   
2. **Initialisation du Tokenizer et du Modèle** : 
   - `tokenizer_c.from_pretrained(weights_pretrained)` et `model_class.from_pretrained(weights_pretrained)` permettent de charger respectivement le tokenizer et le modèle préentraînés. Ces éléments facilitent la tokenisation des textes et la génération de représentations numériques des articles.


In [7]:
model_class, tokenizer_c, weights_pretrained = (transf.DistilBertModel, transf.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_c.from_pretrained(weights_pretrained)
model = model_class.from_pretrained(weights_pretrained)

### Tokenisation des Textes

Cette cellule applique le tokenizer aux textes des articles. 

1. **Tokenisation des articles** : 
   - La fonction `tokenizer.encode()` convertit chaque texte en une séquence d'IDs de tokens en ajoutant des `tokens spéciaux` qui indiquent les débuts et fins des séquences.
   
Le résultat est une série de listes d'IDs de tokens, qui représentent chaque article sous une forme compatible avec le modèle DistilBERT.


In [8]:
tokenized = combined_news_df['text'].apply((lambda x : tokenizer.encode(x, add_special_tokens= True)))

In [9]:
tokenized

0       [101, 13350, 4722, 5998, 1999, 2028, 12528, 10...
1       [101, 2899, 1006, 26665, 1007, 1011, 1996, 205...
2       [101, 6921, 1006, 26665, 1007, 1011, 1996, 139...
3       [101, 2023, 2003, 1037, 2307, 5107, 2000, 3745...
4       [101, 2899, 1006, 26665, 1007, 1011, 1057, 101...
                              ...                        
1353    [101, 7211, 1006, 26665, 1007, 1011, 2822, 234...
1354    [101, 1006, 26665, 1007, 1011, 10210, 2102, 19...
1355    [101, 2899, 1006, 26665, 1007, 1011, 3951, 520...
1356    [101, 27084, 4048, 1010, 3607, 1006, 26665, 10...
1357    [101, 20312, 1006, 26665, 1007, 1011, 4977, 23...
Name: text, Length: 1358, dtype: object

### Calcul de la Longueur Maximale des Séquences

Nous trouvons ici la longueur maximale des séquences tokenisées. 

1. **Calcul de `max_len`** : 
   - En parcourant chaque séquence tokenisée, nous obtenons la longueur maximale. 
   - Cette longueur est utile pour normaliser toutes les séquences à la même taille, en remplissant les séquences plus courtes avec des zéros.

Cela garantit que toutes les séquences ont la même longueur, ce qui est une exigence pour le traitement par les modèles de Transformers.


In [10]:
max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)

max_len

233

### Remplissage des Séquences (Padding)

Dans cette cellule, nous appliquons le remplissage (padding) aux séquences de tokens.

1. **Ajout de zéros** : 
   - Chaque séquence est remplie de zéros à la fin, jusqu'à atteindre `max_len`. 
   - Cela garantit que toutes les séquences ont la même longueur et permet un traitement efficace par le modèle.

Nous utilisons `np.array()` pour convertir le résultat en un tableau NumPy, adapté pour les opérations par lots.


In [27]:
tokenized_zeroes = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [12]:
np.array(tokenized_zeroes).shape

(1358, 233)

### Création du Masque d'Attention

Nous créons ici le masque d’attention, qui indique les tokens réels et les tokens de remplissage.

1. **Génération du masque** : 
   - `np.where(tokenized_zeroes != 0, 1, 0)` crée un masque où chaque token réel est marqué par `1` et chaque token de remplissage par `0`.
   
Le masque d’attention informe le modèle de ne pas prendre en compte les tokens de remplissage lors de la génération des embeddings.


In [13]:
attention_mask = np.where(tokenized_zeroes != 0, 1, 0)
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [14]:
attention_mask.shape

(1358, 233)

### Conversion en Tenseurs PyTorch

Cette cellule convertit les séquences et le masque d’attention en tenseurs PyTorch. 

1. **Conversion des tokens et du masque en tenseurs** : 
   - `torch.tensor()` transforme les tableaux NumPy en objets PyTorch, permettant ainsi leur passage au modèle pour le traitement. 

Les tenseurs `input_ids` et `attention_mask` sont ainsi prêts pour le modèle.


In [15]:
input_ids = torch.tensor(tokenized_zeroes)
attention_mask = torch.tensor(attention_mask)

In [16]:
input_ids

tensor([[  101, 13350,  4722,  ...,     0,     0,     0],
        [  101,  2899,  1006,  ...,     0,     0,     0],
        [  101,  6921,  1006,  ...,     0,     0,     0],
        ...,
        [  101,  2899,  1006,  ...,     0,     0,     0],
        [  101, 27084,  4048,  ...,     0,     0,     0],
        [  101, 20312,  1006,  ...,     0,     0,     0]], dtype=torch.int32)

### Extraction des États Cachés du Modèle

Dans cette cellule, nous faisons passer les tenseurs à travers le modèle pour extraire les états cachés.

1. **Passage par le modèle DistilBERT** : 
   - Avec `torch.no_grad()`, nous désactivons la sauvegarde des gradients, économisant ainsi de la mémoire.
   - `model(input_ids, attention_mask=attention_mask)` retourne les représentations de chaque token dans chaque séquence, soit le dernier état caché pour chaque token.

Les états cachés obtenus servent de caractéristiques de haut niveau pour chaque article.


In [17]:
with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask = attention_mask)

### Sélection des Caractéristiques du CLS Token

Nous extrayons ici les caractéristiques du token `[CLS]` pour chaque séquence. 

1. **Sélection du premier token** :
   - Dans les modèles de Transformers, le token `[CLS]` (première position) contient une représentation de l’ensemble de la séquence, utile pour la classification.
   - `features = last_hidden_states[0][:,0,:]` extrait ces caractéristiques pour chaque article.

Ces caractéristiques seront utilisées comme vecteurs d’entrée pour la classification.


In [18]:
features = last_hidden_states[0][:,0,:]

In [19]:
features.shape

torch.Size([1358, 768])

### Extraction des Labels


Cette cellule extrait les labels associés aux articles dans `combined_news_df`. 

1. **Labeling des caractéristiques** : 
   - `labels = combined_news_df["label"]` récupère les étiquettes `0` ou `1` indiquant si chaque article est faux ou vrai.
   
Ces labels seront utilisés pour entraîner et évaluer notre modèle de classification.


In [20]:
labels = combined_news_df["label"]

### Division des Données en Enseignement et Test

Nous divisons ici les données en ensembles d’apprentissage et de test.

1. **Séparation aléatoire des ensembles** :
   - `train_test_split(features, labels)` crée deux ensembles : un pour l’entraînement (`train_features`, `train_labels`) et un pour le test (`test_features`, `test_labels`). 

La séparation permet d’évaluer la performance du modèle sur des données non vues pendant l’entraînement.


In [21]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

###  Entraînement d'un Classifieur Logistique

Dans cette cellule, nous entraînons un modèle de régression logistique sur les données d’entraînement.

1. **Entraînement** :
   - `lr_clf.fit(train_features, train_labels)` ajuste le modèle de régression logistique en utilisant les vecteurs de caractéristiques extraits du modèle DistilBERT.
   
Ce modèle de régression logistique servira de classificateur pour prédire si un article est vrai ou faux.


In [22]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

### Évaluation de Précision du Modèle

Cette cellule évalue la précision du modèle de régression logistique sur l’ensemble de test.

1. **Calcul de précision** :
   - `lr_clf.score(test_features, test_labels)` mesure la précision du modèle sur les données de test.
   
La précision obtenue donne un aperçu de la capacité du modèle à généraliser sur de nouvelles données.


In [23]:
lr_clf.score(test_features, test_labels)

0.9970588235294118

### Prédiction sur un Nouvel Exemple

Ici, nous testons notre pipeline complet sur un nouvel exemple.

1. **Prétraitement et prédiction** :
   - `tokenizer.encode()` et `padding` préparent le texte pour le modèle.
   - Nous passons les données préparées par le modèle et obtenons `features` (le vecteur `[CLS]`).
   - `lr_clf.predict(features)` retourne la prédiction et `lr_clf.predict_proba(features)` fournit les probabilités pour chaque classe.

Ce test montre comment le modèle prédit la probabilité d'appartenance d’un nouvel article à la classe "vrai" ou "faux".


In [24]:
text= tokenizer.encode("jossua is beautifull", add_special_tokens= True)
tokenized_zeroes = np.array([text + [0]*(max_len - len(text))])
input_ids = torch.tensor(tokenized_zeroes)
attention_mask = torch.tensor(np.where(tokenized_zeroes != 0, 1, 0))
with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask = attention_mask)
features = last_hidden_states[0][:,0,:]

In [25]:
lr_clf.predict(features)

array([0], dtype=int64)

In [26]:
lr_clf.predict_proba(features)

array([[9.99258559e-01, 7.41440974e-04]])

## Conclusion

En conclusion, ce projet démontre l'efficacité du modèle DistilBERT pour la tâche de classification des fake news. Nous avons obtenu des résultats solides en utilisant les caractéristiques extraites par le modèle Transformer, qui a permis de transformer chaque article en un vecteur de haute dimension, facilement exploitable par un modèle de régression logistique pour la classification.

En comparaison avec notre approche précédente utilisant des réseaux de type LSTM, les résultats obtenus avec DistilBERT sont sensiblement similaires en termes de précision et de capacité de généralisation. Cette observation est intéressante car elle suggère que, bien que les modèles de type Transformer comme DistilBERT soient réputés pour leurs performances exceptionnelles dans de nombreuses tâches NLP, les LSTM, une approche plus ancienne, peuvent offrir une performance comparable dans ce cas précis.

Cela peut être dû à plusieurs facteurs, notamment le fait que notre tâche de classification est relativement simple et que la longueur des textes n'est pas très longue dans le cas voulu, ce qui ne justifie peut-être pas pleinement la puissance de DistilBERT. De plus, les LSTM sont capables de capter les relations temporelles et séquentielles dans les données, ce qui peut être avantageux dans des contextes où les structures de texte complexes sont essentielles.

En résumé, bien que DistilBERT soit un modèle plus moderne et souvent plus performant pour des tâches de compréhension de texte complexes, les LSTM, lorsqu'ils sont bien entraînés, peuvent donner des résultats similaires pour des tâches comme la détection de fake news. Le choix du modèle peut donc être guidé par des contraintes pratiques telles que les ressources de calcul disponibles ou la taille des données à traiter.
